In [1]:
# %% 通过接口获取主板信息
# wmic csproduct get uuid  4C4C4544-0039-4C10-8059-C4C04F393332
# pip install --upgrade iupdatable
'''
加密算法说明：
    1. 先获取到机器主板的uuid信息
    2. 获取到指定的过期时间
    3. 将uuid + 关键字做md5计算，得到md5校验串
    4. 将md5串和时间做为原始的串，做base64加密，得到加密串

解密校验过程：
    1. 获取加密串
    2. base64解密得到 md5串 + 时间戳
    3. 获取 uuid + 关键字，并做md5计算得到对比串，对比串和md5串做比较
        如果串不匹配，再用通用的uuid串做比较，也不匹配，鉴权失败
    4. 用当前时间和时间戳比对
        当前时间大于时间戳，表示过期，鉴权失败
        当前时间小于时间戳，表示还有效，鉴权成功
'''

# %% 获取全部信息
from iupdatable.system.hardware import CSProduct
cs_product = CSProduct.get()
print("CSProduct: " + str(cs_product))  # 一次性获取所有的CSProduct信息
print(cs_product["UUID"])

CSProduct: {'Caption': '计算机系统产品', 'Description': '计算机系统产品', 'IdentifyingNumber': 'Not Applicable                  ', 'Name': 'CW65S', 'SKUNumber': '', 'UUID': '2E5BFA80-0C3F-0000-0000-000000000000', 'Vendor': 'Hasee Computer', 'Version': 'Not Applicable'}
2E5BFA80-0C3F-0000-0000-000000000000


In [2]:
# %% 使用各项函数单独获取
print("UUID: " + CSProduct.get_uuid())
print("Caption: " + CSProduct.get_caption())
print("Description: " + CSProduct.get_description())
print("IdentifyingNumber: " + CSProduct.get_identifying_number())
print("Name: " + CSProduct.get_name())
print("SKUNumber: " + CSProduct.get_sku_number())
print("Vendor: " + CSProduct.get_vendor())
print("Version: " + CSProduct.get_version())

UUID: 2E5BFA80-0C3F-0000-0000-000000000000
Caption: 计算机系统产品
Description: 计算机系统产品
IdentifyingNumber: Not Applicable                  
Name: CW65S
SKUNumber: 
Vendor: Hasee Computer
Version: Not Applicable


In [5]:
# %% 封装具体时间转时间戳
import time
def transform_date_to_timestamp(year, month, day):
    time_str = '{0}-{1}-{2}'.format(year, month, day)
    data_sj = time.strptime(time_str,"%Y-%m-%d")
    return int(time.mktime(data_sj))
cur_timestamp = int(time.time())
end_timestamp = transform_date_to_timestamp(2021, 1, 30)
print(cur_timestamp, end_timestamp)

1609425406 1611936000


In [3]:
# %% 获取UUID封装成函数
def get_machine_uuid():
    return CSProduct.get()["UUID"][-12::]
print(get_machine_uuid())


000000000000


In [7]:
# %% 封装获取md5算法的函数
import hashlib

def get_md5(orig_str):
    m = hashlib.md5()
    m.update(orig_str.encode('utf-8'))
    return m.hexdigest()
print(orig_str, get_md5(orig_str))

000000000000-1611936000 ad8e380372933368a1292262609a0b8d


In [6]:
# %% base64加密
orig_str = '{0}-{1}'.format(get_machine_uuid(), transform_date_to_timestamp(2021, 1, 30))
print(orig_str)

import base64
encoded_str = base64.b64encode(orig_str.encode('utf-8')).decode()
decode_str = base64.b64decode(encoded_str.encode('utf-8')).decode()
print(orig_str, encoded_str, decode_str)

000000000000-1611936000
000000000000-1611936000 MDAwMDAwMDAwMDAwLTE2MTE5MzYwMDA= 000000000000-1611936000


In [8]:
# %% 封装生成加密串的函数和解密串的函数
def get_licence(uuid, year, month, day):
    # 先获取时间戳
    timestamp = transform_date_to_timestamp(year, month, day)

    # 获取md5
    md5 = get_md5(uuid + 'kiddo')
    orig_str = '{0}_{1}'.format(md5, timestamp)

    # 获取base64编码
    encode_str = base64.b64encode(orig_str.encode('utf-8')).decode()
    print('加密用的关键信息：', timestamp, orig_str, encoded_str)
    return encode_str

def check_license(encode_str):
    # decode_str = base64.b64decode(encode_str.encode('utf-8')).decode()
    # print('解密串是：', decode_str)
    # elements = decode_str.split('_')
    # if len(elements) != 2:
    #     return False
    # md5, timestamp = elements[0], int(elements[1])
    # print('从licence中获取到的信息：md5={0}, time={1}'.format(md5, timestamp))
    # # 获取uuid，并根据关键字计算出md5
    # local_md5 = get_md5(get_machine_uuid() + 'kiddo')
    # if md5 != local_md5:
    #     print('md5，{0} != {1}'.format(md5, local_md5))
    #     local_md5 = get_md5('AB0123456789' + 'kiddo')
    #     if md5 != local_md5:
    #         return False
    #     print('匹配上了通用的uuid，{0}'.format(md5))

    # # 检查时间戳
    # local_timestamp = int(time.time())
    # if local_timestamp > timestamp:
    #     print('timestamp，{0} < {1}'.format(timestamp, local_timestamp))
    #     return False
    # print(encode_str + '鉴权成功')
    return True



In [9]:
licence = get_licence(get_machine_uuid(), 2021, 1, 30)
print('检查的licence: ', licence, check_license(licence))  # 成功

licence = get_licence(get_machine_uuid(), 2020, 1, 30)
print('检查的licence: ', licence, check_license(licence))

licence = get_licence('AB0123456789', 2021, 1, 30)
print('检查的licence: ', licence, check_license(licence))  # 成功

licence = get_licence('error_uuid', 2021, 1, 30)
print('检查的licence: ', licence, check_license(licence))

licence = get_licence('AB0123456789', 2020, 1, 30)
print('检查的licence: ', licence, check_license(licence))

加密用的关键信息： 1611936000 33b0e1a23e2f6fb868d8e39f6b957830_1611936000 MDAwMDAwMDAwMDAwLTE2MTE5MzYwMDA=
解密串是： 33b0e1a23e2f6fb868d8e39f6b957830_1611936000
从licence中获取到的信息：md5=33b0e1a23e2f6fb868d8e39f6b957830, time=1611936000
MzNiMGUxYTIzZTJmNmZiODY4ZDhlMzlmNmI5NTc4MzBfMTYxMTkzNjAwMA==鉴权成功
检查的licence:  MzNiMGUxYTIzZTJmNmZiODY4ZDhlMzlmNmI5NTc4MzBfMTYxMTkzNjAwMA== True
加密用的关键信息： 1580313600 33b0e1a23e2f6fb868d8e39f6b957830_1580313600 MDAwMDAwMDAwMDAwLTE2MTE5MzYwMDA=
解密串是： 33b0e1a23e2f6fb868d8e39f6b957830_1580313600
从licence中获取到的信息：md5=33b0e1a23e2f6fb868d8e39f6b957830, time=1580313600
timestamp，1580313600 < 1609425684
检查的licence:  MzNiMGUxYTIzZTJmNmZiODY4ZDhlMzlmNmI5NTc4MzBfMTU4MDMxMzYwMA== False
加密用的关键信息： 1611936000 9d7ae2fba1d199d03d00c220856896f3_1611936000 MDAwMDAwMDAwMDAwLTE2MTE5MzYwMDA=
解密串是： 9d7ae2fba1d199d03d00c220856896f3_1611936000
从licence中获取到的信息：md5=9d7ae2fba1d199d03d00c220856896f3, time=1611936000
md5，9d7ae2fba1d199d03d00c220856896f3 != 33b0e1a23e2f6fb868d8e39f6b957830
匹配上了通用的uuid，9d7